In [3]:
# 載入 package
import pandas as pd
import numpy as np
import nltk
import string
import math
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
#nltk.download('punkt')

In [4]:
data = pd.read_csv("yelp.csv")
var = ["stars","text"]
data_pro = data[var]

In [5]:
for i in range(len(data_pro)):
    if (data_pro.iloc[i,0] >= 4):
        data_pro.iloc[i,0] = 1
    else:
        data_pro.iloc[i,0] = 0
#data_pro['stars']=np.where(data_pro['stars']>=4,1,0)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [6]:
train_list = [row[1] for row in data_pro.itertuples(index=False, name=None)]
for i in range(len(data_pro)):
    train_list[i] = train_list[i].translate(str.maketrans('', '', string.punctuation))
#標點符號去除

In [7]:
#設定stop words
stop_words = set(stopwords.words('english'))
ano_stop_words = {'\n','\n\n', 'n', 'cant', 'Ive'}
stop_words = set.union(stop_words, ano_stop_words)

In [8]:
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=10000) # strip_accents='ascii', ngram_range=(1,2), max_df=0.7
vectorizer.fit_transform(train_list)
vectorizer.get_feature_names()
text2vec = vectorizer.fit_transform(train_list).toarray()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ive'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [9]:
tfidf = pd.DataFrame(np.array(text2vec),columns=vectorizer.get_feature_names())
label = data_pro['stars']
#data_final = pd.concat([tfidf,label], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(tfidf, label, train_size=0.8)
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [11]:
from keras.models import Sequential  #用來啟動 NN
from keras.layers import Conv1D  # Convolution Operation
from keras.layers import MaxPooling1D # Pooling
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense # Fully Connected Networks
from keras.layers import LSTM

In [136]:
# define model
model = Sequential()
#model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=8, kernel_size=5, activation='relu', input_shape=(10000,1), padding = 'same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.7))

model.add(Conv1D(filters=8, kernel_size=5, activation='relu', padding = 'same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.7))

model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [137]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [138]:
train_history = model.fit(x_train, y_train, epochs=10, verbose=2, validation_data = (x_test, y_test))

Epoch 1/10
250/250 - 20s - loss: 0.5548 - accuracy: 0.7220 - val_loss: 0.4772 - val_accuracy: 0.7680 - 20s/epoch - 81ms/step
Epoch 2/10
250/250 - 21s - loss: 0.4538 - accuracy: 0.7901 - val_loss: 0.4345 - val_accuracy: 0.8055 - 21s/epoch - 82ms/step
Epoch 3/10
250/250 - 22s - loss: 0.4159 - accuracy: 0.8099 - val_loss: 0.4238 - val_accuracy: 0.8095 - 22s/epoch - 88ms/step
Epoch 4/10
250/250 - 23s - loss: 0.4162 - accuracy: 0.8117 - val_loss: 0.4311 - val_accuracy: 0.8145 - 23s/epoch - 91ms/step
Epoch 5/10
250/250 - 22s - loss: 0.4075 - accuracy: 0.8096 - val_loss: 0.4241 - val_accuracy: 0.8095 - 22s/epoch - 87ms/step
Epoch 6/10
250/250 - 24s - loss: 0.3931 - accuracy: 0.8256 - val_loss: 0.4184 - val_accuracy: 0.8065 - 24s/epoch - 96ms/step
Epoch 7/10
250/250 - 24s - loss: 0.3798 - accuracy: 0.8303 - val_loss: 0.4154 - val_accuracy: 0.8095 - 24s/epoch - 94ms/step
Epoch 8/10
250/250 - 25s - loss: 0.3726 - accuracy: 0.8292 - val_loss: 0.4146 - val_accuracy: 0.8095 - 25s/epoch - 100ms/step

In [16]:
lstm_model = Sequential()
lstm_model.add(LSTM(units = 10, return_sequences = True, input_shape = (10000, 1)))
lstm_model.add(Dropout(0.7))

#lstm_model.add(LSTM(units = 10))
#lstm_model.add(Dropout(0.7))

lstm_model.add(Dense(units = 1))
lstm_model.compile(optimizer = 'adam', loss = 'binary_crossentropy',  metrics = ['accuracy'])

In [17]:
train_history = lstm_model.fit(x_train, y_train, epochs=10, verbose=2, validation_data = (x_test, y_test))

Epoch 1/10
250/250 - 747s - loss: 2.0686 - accuracy: 0.3365 - val_loss: 1.1870 - val_accuracy: 0.3200 - 747s/epoch - 3s/step
Epoch 2/10
250/250 - 742s - loss: 1.0918 - accuracy: 0.4013 - val_loss: 0.7599 - val_accuracy: 0.3200 - 742s/epoch - 3s/step
Epoch 3/10
250/250 - 549s - loss: 0.9912 - accuracy: 0.4359 - val_loss: 0.7544 - val_accuracy: 0.3200 - 549s/epoch - 2s/step
Epoch 4/10


KeyboardInterrupt: 